# Prepare Notebook Environment for Strands Agents

In this section we prepare this notebook environment with the necessary dependencies to create any agents using the Strands framework.

## Prerequisites

1. If you are running the notebook in your own account and not on as part of an AWS hosted event, follow the **Deploy Infrastructure** instructions from the [Self Paced](https://catalog.us-east-1.prod.workshops.aws/workshops/0bf501fc-357d-43b3-b758-b44f9b0e7d49/en-US/access-and-navigate-aws-account/self-paced-agentcore) workshop to setup the following dependencies in your account:
    - Networking infrastructure (VPC, Subnets, etc.)
    - Amazon Redshift Database
    - AWS Step Function
    - Amazon Bedrock Knowledge Base
2. Manually sync the Amazon Bedrock Knowledge Base:
    - Navigate to the Amazon Bedrock dashboard via AWS Console search
    - Select the option icon (top left) to open the navigation bar
    - Select "Knowledge bases" under the "Builder tools" tab
    - Choose the Knowledgebase created by the AWS CloudFormation template
    - Scroll to the "Data Source" option box
    - Select the data source (radio button) and Select Sync
3. Make sure your notebook execution role has the following managed policies (you can find an example at [sagemaker_execution_role.yaml](sagemaker_execution_role.yaml)):
    - AmazonBedrockFullAccess
    - AmazonRedshiftQueryEditor
    - AmazonS3FullAccess
    - AmazonSageMakerFullAccess
    - AWSLambda_FullAccess
    - AWSStepFunctionsFullAccess
    - IAMFullAccess
    - AWSCodeBuildAdminAccess

## Environment Setup

#### Run the pip commands below to install all needed packages

In [ ]:
%pip install boto3==1.37.1 awscli botocore==1.37.1 termcolor==3.1.0 sagemaker==2.245.0 --quiet

Ensure the boto3 version printed below is **1.37.1** or higher.

In [ ]:
%pip show boto3

#### Import Python libraries

In [ ]:
# Standard Python libraries
import boto3
import sagemaker

#### Extract account information needed for agent creation notebook role

In [ ]:
# boto3 session
sts_client = boto3.client('sts')
session = boto3.session.Session()

# Account info
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
print(f"Account ID: {account_id}")
print(f"Region: {region}")

# Get the SageMaker session and execution role
sagemaker_session = sagemaker.Session()
role = sagemaker_session.get_caller_identity_arn()
print(f"Notebook Execution Role: {role}")

## Configure Bedrock AgentCore Observability on Amazon CloudWatch
AgentCore Observability helps you trace, debug, and monitor agent performance in production environments. It offers detailed visualizations of each step in the agent workflow, enabling you to inspect an agent's execution path, audit intermediate outputs, and debug performance bottlenecks and failures.

AgentCore Observability gives you real-time visibility into agent operational performance through access to dashboards powered by Amazon CloudWatch and telemetry for key metrics such as session count, latency, duration, token usage, and error rates. Rich metadata tagging and filtering simplify issue investigation and quality maintenance at scale. AgentCore emits telemetry data in standardized OpenTelemetry (OTEL)-compatible format, enabling you to easily integrate it with your existing monitoring and observability stack.

All of the metrics, spans, and logs output by AgentCore are stored in Amazon CloudWatch, and can be viewed in the CloudWatch console or downloaded from CloudWatch using the AWS CLI or one of the AWS SDKs.

Follow the below steps to enable observability for agents hosted on AgentCore Runtime:
1. Go to **Amazon CloudWatch** on the AWS Console
2. Select **GenAI Observability** under **Application Signals(APM)** on the left side of the navigation pane.
3. Select the **Bedrock AgentCore** tab
4. AgentCore observability require you to enable Transaction Search. To do that, select **Configure** to open a new browser tab with X-Ray traces configuration.

![Configure Transaction Search](../images/observability_setup_1.png)

5. To enable Transaction Search, Select **Edit**, mark the checkbox and select Save. It might take up to 5 minutes for the configuration to go from *Updating* to *Enabled*.

![Configure Transaction Search](../images/observability_setup_2.png)

6. Close the X-Ray tab and go back to **GenAI Observability** view on **Amazon CloudWatch**. You might need to refresh the page if you are still seeing the *Configure* button. You can now explore the different metrics for your agents.